In [1]:
import numpy as np
import spglib

In [2]:
class Document: 
    def __init__(self, filename):
        self.filename = filename
        
    def formatted_poscar(self):
        filename_z = self.filename
        with open(filename_z) as file_object_ls:
            lines_ls = file_object_ls.readlines()
    
        n_atoms = sum([int(i) for i in lines_ls[6].split()]) #number of atoms
    
        lines2 = lines_ls[8:]
        list_ls = [i.split() for i in lines2]
        self.LS= np.array(list_ls).reshape(n_atoms, 3).astype(float)
    
        ls1 = []
        ls2 = []
        ls3 = []
    
        # corrections
        for i in self.LS[:,0]:
            if i >= 0.90:
                i = i - 1
            elif i < -0.90:
                i = i + 1
            else:
                i = i
            ls1.append(i)
        ls1 = np.array(ls1).reshape(n_atoms,1)
    
        for j in self.LS[:,1]:
            if j >= 0.9:
                j = j - 1
            elif j < -0.9:
                j = j + 1
            else:
                j = j
            ls2.append(j)
        ls2 = np.array(ls2).reshape(n_atoms,1)
        ls3 = np.array(self.LS[:,2]).reshape(n_atoms,1)
    
        LS_final = np.hstack([ls1, ls2, ls3]) #corrected final structure
        return LS_final
    
    def lattice_parameters(self):
        with open(self.filename) as file_for_lattice:
            lines = file_for_lattice.readlines()
        
        a = lines[2].split()
        b = lines[3].split()
        c = lines[4].split()
    
        self.lat_a = np.sqrt(sum([float(i)**2 for i in a]))
        self.lat_b = np.sqrt(sum([float(i)**2 for i in b]))
        self.lat_c = np.sqrt(sum([float(i)**2 for i in c]))
        return self.lat_a, self.lat_b, self.lat_c
    
    def volume(self):
        self.volume = self.lat_a * self.lat_b * self.lat_c
        return self.volume
    
    def spacegroup(self):
        filename_y = self.filename
        with open(filename_y) as file_object_ls:
            lines_ls = file_object_ls.readlines()
        atom_n = [int(i) for i in lines_ls[6].split()]
        empt = []
        for i in range(len(atom_n)):
            num = atom_n[i]*[i+1]
            empt.append(num)
        numbers = sum(empt, [])
        
        lattu = lines_ls[2:5]
        latt = [i.split() for i in lattu]
        lattice = []
        for l in latt:
            lattice.append([float(i) for i in l])
            
        n_atoms = sum([int(i) for i in lines_ls[6].split()]) #number of atoms
    
        lines2 = lines_ls[8:]
        list_ls = [i.split() for i in lines2]
        positions = np.array(list_ls).reshape(n_atoms, 3).astype(float)
        
        space = spglib.get_spacegroup(cell=(lattice, positions, numbers), angle_tolerance=-1.0, symbol_type=0)
        
        return "The space group of your structure is {}".format(space)
        
        

In [3]:
myfile = Document('CONTCAR_ls')

In [4]:
myfile.lattice_parameters()

(6.373357338259712, 6.373357338334221, 12.381713439972922)

In [5]:
myfile.spacegroup()

'The space group of your structure is P6_3cm (185)'

In [7]:
myfile.volume()

502.9412843587471